# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
!pip install hvplot


In [ ]:
!pip install opencv-python

In [12]:
import os

In [ ]:
os.getcwd()

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\maclo\\onedrive\\desktop\\Repositories\\python-api\\HW 6\\Starter_Code\\starter_code\\output_data\\cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,New York,US,40.7143,-74.0060,69.51,28,0,9.22,1681148800
1,1,Los Angeles,US,34.0522,-118.2437,87.21,69,40,5.75,1681148838
2,2,Paris,FR,48.8534,2.3488,58.08,92,100,13.80,1681149136
3,3,Tokyo,JP,35.6895,139.6917,61.56,63,0,16.11,1681148886
4,4,Sydney,AU,-33.8679,151.2073,54.63,68,0,9.22,1681148930


In [1]:
!pip install hvplot pandas


In [4]:
!pip install --upgrade pip setuptools


  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 47.3.1.post20200622
    Uninstalling setuptools-47.3.1.post20200622:
      Successfully uninstalled setuptools-47.3.1.post20200622


In [8]:
!pip install wheel
!pip install numpy


In [9]:
!pip install shapely


     ---------------------------------------- 1.4/1.4 MB 4.9 MB/s eta 0:00:00


In [ ]:
!conda install -c conda-forge shapely


In [ ]:
import hvplot.pandas
import pandas as pd
from IPython.display import display

# Configure the map plot
plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width=500,
    frame_height=400,
    geo=True,
    tiles="CartoLight",
    colorbar=True,
    clim=(0, 100),
)

# Display the map
display(plot)


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width=500,
    frame_height=400,
    geo=True,
    tiles="CartoLight",
    colorbar=True,
    clim=(0, 100),
)

# Display the map
hvplot.show(plot)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.query('`Max Temp` >= 280.00 and `Max Temp` <= 300.00 and `Wind Speed` < 1.54 and `Cloudiness` == 0')

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
limit = 20
radius = 10000
params = {
    "categories" : categories,
    "limit":limit,
    "radius" :radius,
    "sort" : "distance",
    "apiKey" :geoapify_key
}
# Set the parameters for the type of search
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filters'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params).json()
    # Convert the API response to JSON format
    name_address = response
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

hover_cols = ['City', 'Country', 'Hotel Name']
hover_tooltips = [(col, '@'+col) for col in hover_cols]

plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width=500,
    frame_height=400,
    geo=True,
    tiles="CartoLight",
    colorbar=True,
    clim=(0, 100),
    hover_cols=hover_cols,
    hover_tooltips=hover_tooltips,
)

# Display the map
hvplot.show(plot)